In [ ]:
%load_ext autoreload
%autoreload 2

import os
ipy = get_ipython()
#data_dir_general = '/home/demitau/data_Quentin'
#data_subdir_mem_err_main = 'full_experiments/data2'
from os.path import join as pjoin
import os,sys; sys.path.append(os.path.expandvars('$CODE_MEMORY_ERRORS'))
import error_sensitivity
#data_dir_input = pjoin(data_dir_general,data_subdir_mem_err_main)
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df_all_multi_tsz = pd.read_pickle('/home/demitau/data_Quentin/full_experiments/data2/df_all_multi_tsz.pkl.zip')

In [ ]:
df_all_multi_tsz = pd.read_pickle('/home/demitau/ownCloud/Current/merr_data/df_all_multi_tsz_NIH.pkl.zip')

In [ ]:
df_all_multi_tsz.columns

In [ ]:
df_all = df_all_multi_tsz.query('trial_shift_size == 1 and trial_group_col_calc == "trials"').copy()

In [ ]:
assert not df_all.duplicated(['subject','trials']).any()

In [ ]:
subjects = df_all['subject'].unique()
print(subjects)

In [ ]:
df_all = df_all.sort_values(['subject','trials'])

vs = ['feedback','prev_feedback','belief','prev_belief','error','prev_error','movement','prev_movement']

vs2 = ['subject','trials'] + vs

df_all['prev_feedback'] = df_all.groupby('subject')['feedback'].shift(1)

df_all['feedback']      = df_all['feedback'] - np.pi
df_all['prev_feedback'] = df_all['prev_feedback'] - np.pi

df_all['movement'] = df_all['org_feedback'] - np.pi
df_all['prev_movement'] = df_all.groupby('subject')['org_feedback'].shift(1) - np.pi

df_all['belief']  = df_all['org_feedback'] - df_all['target_locs']
df_all['prev_belief'] =  df_all.groupby('subject')['belief'].shift(1)

In [ ]:
subjects

In [ ]:
sns.pairplot(df_all.query('environment == 0 and subject ==@subjects[0]')[vs])

In [ ]:
# very corr to 'err' column
df_all['error_pscadj_pertstageadj_abs'] = df_all['error_pscadj_pertstageadj'].abs()

import numpy as np
import pandas as pd
from scipy.stats import pearsonr

def select_numeric_columns(df):
    numerical_columns = []
    for column in df.columns:
        if np.issubdtype(df[column].dtype, np.number):
            numerical_columns.append(column)
    return df[numerical_columns]

def compute_significant_correlations(df, allow_empty=0):
    correlations = []
    for i in range(len(df.columns)):
        for j in range(i + 1, len(df.columns)):
            dfi = df[df.columns[i]]
            bad1 = dfi.isna() | np.isinf(dfi)
            dfj = df[df.columns[j]]
            bad2 = dfj.isna() | np.isinf(dfj)
            subdf = df[~(bad1 | bad2)]
            if allow_empty and len(subdf) == 0:
                corr, pval = None,1
            else:
                corr, pval = pearsonr(subdf[subdf.columns[i]], 
                                      subdf[subdf.columns[j]])
            if pval < 0.05:
                d = dict(zip(['col1','col2','r','pval'],(df.columns[i], 
                    df.columns[j], corr, pval)) )
                
                correlations.append(d)
    return pd.DataFrame(correlations)

# df = pd.DataFrame({
#     "x": np.random.randint(0, 100, 100),
#     "y": np.random.randint(0, 100, 100),
#     "z": np.random.randint(0, 100, 100)
# })

print(len(df_all.query('error_pscadj_pertstageadj_abs >= 1.5 ') ))
df_all_c = df_all.query('error_pscadj_pertstageadj_abs < 1.5 ') 

df_num = select_numeric_columns(df_all_c.drop(columns=['index']))
correlations = compute_significant_correlations(df_num)
display(correlations)


In [ ]:
correlations_st = compute_significant_correlations(df_num.query('environment == 0'))
correlations_rnd = compute_significant_correlations(df_num.query('environment == 1'), allow_empty = 1)

In [ ]:
correlations['rabs'] = correlations['r'].abs()
correlations_st['rabs'] = correlations_st['r'].abs()
correlations_rnd['rabs'] = correlations_rnd['r'].abs()

In [ ]:
len(correlations), len(correlations_st), len(correlations_rnd)

In [ ]:
cols = [col for col in df_all.columns if not col.startswith('trial')]
cols = list( (set(cols) - set(['movement',
     'feedbackX','feedbackY','error_pscadj_pertstageadj',
     'target_locs','err','target_codes','org_feedback_pscadj',
    'error_pscadj_pertstageadj', 'org_feedback'] ) )
            | set(['trials','trialwtgt','trialwpert_wb']) )

In [ ]:
cols

In [ ]:
inds = [3,21, 28,31,155, 180,181,182,183,186,214,215,222, 248, 307, 355] +\
    list(range(40,56+1)) + list(range(249,288+1))

In [ ]:
# see what abserr would be corr with

In [ ]:
correlations.to_pickle('/home/demitau/data_Quentin/full_experiments/data2/NIH_behav_signif_corr_exchugeerr.pkl')
correlations_st.to_pickle('/home/demitau/data_Quentin/full_experiments/data2/'
                       'NIH_behav_stable_signif_corr_exchugeerr.pkl')
correlations_rnd.to_pickle('/home/demitau/data_Quentin/full_experiments/data2/'
                       'NIH_behav_random_signif_corr_exchugeerr.pkl')

In [ ]:
def srt(lst):
    lst = list(lst)
    r = sorted(lst, 
       key=lambda x: list(df_all.columns.values).index(x))
    return tuple(list(r))

In [ ]:
srt(['RT', 'prev_belief']), srt(['prev_belief','RT'])

In [ ]:
sets_st = [srt(l) for l in correlations_st[['col1','col2']].values]
sets_rnd = [srt(l) for l in correlations_rnd[['col1','col2']].values]
sets_all = [srt(l) for l in correlations[['col1','col2']].values]

In [ ]:
(len(sets_all), len(sets_st), len( set(sets_all) ^ set(sets_st) ) ,
 len( set(sets_all) ^ set(sets_rnd ) ), 
 len( set(sets_st) ^ set(sets_rnd) ) )   

In [ ]:
sets_st

In [ ]:
set(sets_st)

In [ ]:
#all
with pd.option_context('display.max_rows', None):
    display(correlations_st.\
            query('col1.isin(@cols) and col2.isin(@cols)') )

In [ ]:
#all
with pd.option_context('display.max_rows', None):
    display(correlations.\
            query('col1.isin(@cols) and col2.isin(@cols)') )

In [ ]:
def f(x):
    #c1,c2 = srt((x['col1'],x['col2']))
    if x['col2'] < x['col1'] or len(x['col2']) > len(x['col1']):
        c1,c2 = x['col2'],x['col1']
    else:
        c1,c2 = x['col1'],x['col2']
    return c1,c2
correlations[['col1srt','col2srt']] = correlations.apply(
    f,1 ,result_type='expand')
correlations_st[['col1srt','col2srt']] = correlations_st.apply(
    f,1 ,result_type='expand')
correlations_rnd[['col1srt','col2srt']] = correlations_rnd.apply(
    f,1 ,result_type='expand')

In [ ]:
df_.columns

In [ ]:
df0 = correlations_st

cols_int = ['error_pscadj_pertstageadj_abs','err_sens',
'error','trialwb','movement_duration',
'RT','perturbation']

col2info = {}
for col_int in cols_int:#[:1]:
    neg = []
    pos = []
    with pd.option_context('display.max_rows', None):
        df_ = df0.query('col1.isin(@cols) and col2.isin(@cols)')
        #df_ = df0
        df_ = df_.query('col2 == @col_int '
                    'or col1 == @col_int')\
            [['col1srt','col2srt','r','pval']].sort_values('r')
        

        #dfneg = df_.query('r < 0')
        display( df_)
    #dfneg = df_.query('r < 0')
    for i,row in df_.iterrows():
        r = row['r']
        othercol = list(set([row['col1srt'],row['col2srt']])-\
                        set([col_int]))[0]
        if r < 0:            
            neg += [(othercol, row['r'], row['pval']) ]
        else:
            pos += [(othercol, row['r'], row['pval'])]
    
    col2info[col_int] = {'pos':pos, 'neg':neg}

for col_int,info in col2info.items():
    neg = [col[0] for col in info['neg'] ]
    pos = [col[0] for col in info['pos'] ]
    print('{} is negatively correlated with {}'.format( col_int,
                                    sorted(neg) ) )
    print('  it is postively correlated with ',sorted(pos))
    print('')

In [ ]:
df0 = correlations_rnd

cols_int = ['error_pscadj_pertstageadj_abs','err_sens',
'error','trialwb','movement_duration',
'RT','perturbation']

col2info = {}
for col_int in cols_int:#[:1]:
    neg = []
    pos = []
    with pd.option_context('display.max_rows', None):
        df_ = df0.query('col1.isin(@cols) and col2.isin(@cols)')
        df_ = df_.query('col2 == @col_int '
                    'or col1 == @col_int')\
            [['col1srt','col2srt','r','pval']].sort_values('r')
        

        #dfneg = df_.query('r < 0')
        display( df_)
    #dfneg = df_.query('r < 0')
    for i,row in df_.iterrows():
        r = row['r']
        othercol = list(set([row['col1srt'],row['col2srt']])-\
                        set([col_int]))[0]
        if r < 0:            
            neg += [(othercol, row['r'], row['pval']) ]
        else:
            pos += [(othercol, row['r'], row['pval'])]
    
    col2info[col_int] = {'pos':pos, 'neg':neg}

for col_int,info in col2info.items():
    neg = [col[0] for col in info['neg'] ]
    pos = [col[0] for col in info['pos'] ]
    print('{} is negatively correlated with {}'.format( col_int,
                                    sorted(neg) ) )
    print('  it is postively correlated with ',sorted(pos))
    print('')

In [ ]:
#all
#col_int = 'error_pscadj_pertstageadj_abs'
#col_int = 'trialwb'
#col_int = 'err_sens'
#col_int = 'movement_duration'
col_int = 'RT'
with pd.option_context('display.max_rows', None):
    display(correlations.query('col2 == @col_int or col1 == @col_int')\
            [['col1','col2','r','pval']].sort_values('r') )

In [ ]:
#all
with pd.option_context('display.max_rows', None):
    display(correlations.loc[inds] [['col1','col2','r','pval']] )

In [ ]:
df_all_c.plot(x='trials',y=['error_pscadj_pertstageadj_abs'])

In [ ]:
df_all['pert_stage_wb'].max()#,df_all['pert_stage_'].max()

In [ ]:
df_all[['trials','trialwtgt']]

In [ ]:
dfos = df_all.query('environment == 0 and subject ==@subjects[0]')
dfos = dfos[~dfos['error'].isna() & ~dfos['prev_error'].isna()]

In [ ]:
from scipy.stats import pearsonr
#np.corrcoef
pearsonr(dfos['error'], dfos['prev_error'])

In [ ]:
pearsonr(dfos['movement'], dfos['prev_movement'])

In [ ]:
sns.pairplot(df_all_c.query('subject ==@subjects[0]')[vs])

In [ ]:
df_all['pert_stage_wb'].max()

In [ ]:
sns.pairplot(df_all_c.query(
    'environment == 0 and prev_error.abs() < 1.6')[vs])

sns.pairplot(df_all_c.query(
    'environment == 0 and prev_error.abs() < 1.6')[['error','prev_error']])

In [ ]:
#halfpi = np.pi / 2

In [ ]:
sns.pairplot(df_all_c.query(
    'environment == 1 and belief.abs() < 1.6 and '
     ' prev_error.abs() < 1.6'
    ' and prev_belief.abs() < 1.6')[vs])

In [ ]:
sns.pairplot(df_all_c.query(
    'environment == 1 and belief.abs() < 1.6 and '
     ' prev_error.abs() < 1.6'
    ' and prev_belief.abs() < 1.6')[['error','prev_error']])

In [ ]:
sns.pairplot(df_all[vs])

In [ ]:
import numpy as np
(df_all['prev_belief']*180/np.pi).hist()

In [ ]:
import numpy as np
(df_all['movement']*180/np.pi).hist()

In [ ]:
df_all['movement'].describe()

In [ ]:
df_all['movement'].min()

In [ ]:
df_all['error'].min()

In [ ]:
df_noc_st = df_all.query('environment == 0')[vs2]
df_c_st = df_noc_st.copy()
df_c_st.loc[:,vs] = df_c_st.loc[:,vs].apply(lambda x: x-x.mean(), axis=0)

df_noc = df_all[vs2]
df_c = df_noc.copy()
df_c.loc[:,vs] = df_c.loc[:,vs].apply(lambda x: x-x.mean(), axis=0)

In [ ]:
from scipy.stats import pearsonr
#np.corrcoef
for df_ in [df_noc_st, df_c_st]:
    dfg = df_.query('subject ==@subjects[0]')
    dfg = dfg[~dfg['error'].isna() & ~dfg['prev_error'].isna()]
    c1 = pearsonr(dfg['error'], dfg['prev_error'])
    c2 = pearsonr(dfg['movement'], dfg['prev_movement'])
    print(c1,c2)

In [ ]:
df_ = df_c_st[vs]
(df_/df_.std()).cov()
#.cov()

In [ ]:
display( df_noc[vs].describe().loc[['count','mean','std']] )
display( df_c[vs].describe().loc[['count','mean','std']] )

In [ ]:
df_noc[vs] - df_c[vs]

In [ ]:
#df_c

In [ ]:
import gc; gc.collect()

In [ ]:
df_ = df_noc[vs]
df_norm = df_ / df_.std()

In [ ]:
df_noc[vs].isna().sum()

In [ ]:
df_noc[vs].interpolate(method='linear').dropna().isna().sum()

In [ ]:
# Compute the correlation matrix using the dot product of the normalized dataframe and its transpose
df_norm.T.dot(df_norm)
# Print the correlation matrix

In [ ]:
def dumbCorr(df):
    df = df.interpolate(method='linear').dropna()
    df_norm = df / df.std()
    # Compute the correlation matrix using the dot product of the normalized dataframe and its transpose
    corr_matrix = df_norm.T.dot(df_norm)
    # Print the correlation matrix
    return corr_matrix
#print(corr_matrix)

In [ ]:
ddof = None
cov_matrix_noc = df_noc[vs].cov(ddof=ddof)
#plot the heatmap of the covariance matrix
ax = sns.heatmap(cov_matrix_noc)
ax.invert_yaxis()
ax.set_title('pandas cov UNcentered')
plt.figure()
cov_matrix_c = df_c[vs].cov(ddof = ddof)
#plot the heatmap of the covariance matrix
ax = sns.heatmap(cov_matrix_c)
ax.invert_yaxis()
ax.set_title('pandas cov centered')

plt.figure()
corr_matrix_noc = df_noc[vs].corr()
ax = sns.heatmap(corr_matrix_noc)
ax.invert_yaxis()
ax.set_title('pandas corr UNcentered')
plt.figure()
corr_matrix_c = df_c[vs].corr()
ax = sns.heatmap(corr_matrix_c)
ax.invert_yaxis()
ax.set_title('pandas corr centered')

plt.figure()
dcorr_matrix_noc = dumbCorr(df_noc[vs])
ax = sns.heatmap(dcorr_matrix_noc)
ax.invert_yaxis()
ax.set_title('dumb corr UNcentered')
plt.figure()
dcorr_matrix_c = dumbCorr(df_c[vs])
ax = sns.heatmap(dcorr_matrix_c)
ax.invert_yaxis()
ax.set_title('dumb corr centered')

In [ ]:
df_noc, df_c = \
    df_noc_st.query('subject == @subjects[0]'), \
    df_c_st.query('subject == @subjects[0]')

In [ ]:
df_noc_, df_c_ = df_noc_st, df_c_st

In [ ]:
ddof = 1
cov_matrix_noc = df_noc_[vs].cov(ddof=ddof)
#plot the heatmap of the covariance matrix
ax = sns.heatmap(cov_matrix_noc)
ax.invert_yaxis()
ax.set_title('pandas cov UNcentered')
plt.figure()
cov_matrix_c = df_c_[vs].cov(ddof = ddof)
#plot the heatmap of the covariance matrix
ax = sns.heatmap(cov_matrix_c)
ax.invert_yaxis()
ax.set_title('pandas cov centered')

plt.figure()
cov_matrix_noc = (df_noc_/df_noc_.std())[vs].cov(ddof=ddof)
#plot the heatmap of the covariance matrix
ax = sns.heatmap(cov_matrix_noc)
ax.invert_yaxis()
ax.set_title('pandas normalized cov UNcentered')
plt.figure()
cov_matrix_c = (df_c_/df_c_.std())[vs].cov(ddof = ddof)
#plot the heatmap of the covariance matrix
ax = sns.heatmap(cov_matrix_c)
ax.invert_yaxis()
ax.set_title('pandas normalized cov centered')

plt.figure()
corr_matrix_noc = df_noc_[vs].corr()
ax = sns.heatmap(corr_matrix_noc)
ax.invert_yaxis()
ax.set_title('pandas corr UNcentered')
plt.figure()
corr_matrix_c = df_c_[vs].corr()
ax = sns.heatmap(corr_matrix_c)
ax.invert_yaxis()
ax.set_title('pandas corr centered')

plt.figure()
dcorr_matrix_noc = dumbCorr(df_noc_[vs])
ax = sns.heatmap(dcorr_matrix_noc)
ax.invert_yaxis()
ax.set_title('dumb corr UNcentered')
plt.figure()
dcorr_matrix_c = dumbCorr(df_c_[vs])
ax = sns.heatmap(dcorr_matrix_c)
ax.invert_yaxis()
ax.set_title('dumb corr centered')

In [ ]:
dcorr_matrix_noc - dcorr_matrix_c

In [ ]:
#If rowvar is True (default), then each row represents a variable, with observations in the columns.

In [ ]:
cov_matrix_noc - cov_matrix_c

In [ ]:
ma.cov?

In [ ]:
import numpy.ma as ma
marr = ma.masked_invalid(df_noc[vs].values.T)
#compute the covariance using numpy.ma.cov
cov_ = ma.cov(marr)
cov_noc = cov_.filled(np.nan)
assert not np.isnan(cov_noc).any()

marr = ma.masked_invalid(df_c[vs].values.T)
#compute the covariance using numpy.ma.cov
cov_ = ma.cov(marr)
cov_c = cov_.filled(np.nan)
assert not np.isnan(cov_c).any()

In [ ]:
np.abs(cov_c - cov_noc).max()

In [ ]:
vs

In [ ]:
#cov.shape

In [ ]:
import pingouin
from pingouin import partial_corr
pingouin.__version__

In [ ]:
vs

In [ ]:
vs0 = [ 'error',
 'prev_error',
       'prev_belief',
 'prev_movement']
susp_var = 'prev_movement'


In [ ]:
from base2 import B2B
from sklearn.linear_model import LinearRegression
def partial_corr_b2b(df, x,y, covar, intercept_G = 0, intercept_H = 0):
    #X,Y
    df = df.interpolate(method='linear').dropna()
    X = df[x].values[:,None]
    Y = df[[y] + covar].values
    #print(Y.shape)
    
    G = LinearRegression(fit_intercept = bool(intercept_G) )
    H = LinearRegression(fit_intercept = bool(intercept_H) )
    b2b = B2B(G,H, random_state_split = 0)
    b2b.fit(X,Y, verbose=False)
    return np.diag(b2b.E_), b2b
r, b2b = partial_corr_b2b(df_c, susp_var, v1, other_vars)

In [ ]:
b2b

In [ ]:
rs = []
for susp_var in vs0:
    for name,df in [('c',df_c),('noc',df_noc)]:
        for v1 in (set(vs0) - set([susp_var])):
            for alt in ['two-sided','greater','less']:
                other_vars = list(set(vs0) - set([susp_var,v1]))#; print(other_vars)
                r = partial_corr(df, x = susp_var, y=v1, covar=other_vars, alternative=alt)
                r['alt'] = alt
                r['v1'] = v1
                r['centering'] = name
                r['susp_var'] = susp_var
                r['covar']  = ','.join(list(sorted(other_vars) ))
                r['alg'] = 'partial_corr'
                r['iG'] = 0
                r['iH'] = 0
                rs += [r]
            
            for (iG,iH) in [(0,0), (1,1), (1,0), (0,1)]:
                np.random.seed(0)
                r = partial_corr_b2b(df, susp_var, v1, other_vars, iG,iH)
                d = { 'r': r[0], 'r1': r[1], 'r2': r[2]}
                r = pd.DataFrame([d])
                r['alt'] = alt
                r['v1'] = v1
                r['centering'] = name
                r['susp_var'] = susp_var
                r['covar']  = ','.join(list(sorted(other_vars) ))
                r['alg'] = 'partial_corr_b2b'
                r['iG'] = iG
                r['iH'] = iH
                rs += [r]
dfr= pd.concat(rs).sort_values(['susp_var','v1','alt','alg','centering']).reset_index(drop=1)
dfr

In [ ]:
#def f(dftmp):
#    dftmp.query('centering')
dfr_ = dfr.query('alg == "partial_corr"')
dif = dfr_.query('centering == "c"').set_index(['susp_var','v1','alt'])['r'] - \
    dfr_.query('centering == "noc"').set_index(['susp_var','v1','alt'])['r']
(dif.abs() > 1e-10).any()

In [ ]:
#def f(dftmp):
#    dftmp.query('centering')
dfr_ = dfr.query('alg == "partial_corr_b2b"')
dif = dfr_.query('centering == "c"').set_index(['susp_var','v1','iG','iH'])['r'] - \
    dfr_.query('centering == "noc"').set_index(['susp_var','v1','iG','iH'])['r']
(dif.abs() > 1e-10).any()

In [ ]:
for (iG,iH) in [(0,0), (1,1), (1,0), (0,1)]:
    print(iG,iH, 'max abs dif = {:.6f}'.format( dif[:,:,iG,iH].abs().max() ) )

In [ ]:
dif.groupby(['iG','iH']).abs()

In [ ]:
with pd.option_context('display.max_rows', None,):
    df_ = dfr.query('alg == "partial_corr_b2b" and iG == iH and iH == 1')[['susp_var','r1','covar', 'centering','r','r1','r2','iG','iH']]
    display(df_)

In [ ]:
# pingouin.partial_corr(data=None, x=None, y=None, covar=None, x_covar=None, y_covar=None, alternative='two-sided', method='pearson')
# Partial and semi-partial correlation.

